# **Performance Evaluation Agent**

To evaluate the performance of our recommender system, we will analyze the system regarding the individual agents’ performance, the cold start problem, the sensitivity to changes in the recommendation hyperparameter and potential energy cost savings. We will introduce a further agent called Evaluation Agent which will perform all evaluation actions.

We will perform our evaluation analysis for households 1 to 10 in the REFIT: Electrical Load Measurements data (Murray et at., 2019), to validate our evaluation results.


## **1. Preparing the Environment**

In [ ]:
from agents import Performance_Evaluation_Agent
from helper_functions import Helper

helper = Helper()

import pandas as pd
import numpy as np
import json
import pickle
from copy import deepcopy
import time
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
#import statsmodels.api as sm

import scipy.spatial

In [ ]:
DATA_PATH = '../data/'
EXPORT_PATH = '../export/'

<br>

## **2. Preparations for Evaluating the Performance of our Recommender System**
### **2.1 Determining User Input**

Before we are able to use our recommender system and evaluate its performance, we need to specify the required user inputs (i.e. active appliances, shiftable devices and the consumption threshold). For specifying which of the devices in the respective household will be determined as active appliances and shiftable devices, we look at the description of the devices provided in the readme file and categorize the devices according to our definitions of the categories. Furthermore, we validate that the used devices do not contain any noise in their consumption data and remove devices which contain noise, i.e. consume energy constantly over time. 

In [ ]:
readme = DATA_PATH+'REFIT_Readme.txt'
readme = helper.load_txt(readme)
start = readme.rfind('House 1\n')
end = readme.find('House 11\n')
print(readme[start:end])

<br>

**Shiftable Devices**

In [ ]:
# manual input, determined using information provided in the readme
# validated in the next step using the validate thresholds functionality
shiftable_devices = {
    1: ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
    2: ['Washing Machine', 'Dishwasher'],
    3: ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
    4: ['Washing Machine (1)', 'Washing Machine (2)'],
    5: ['Tumble Dryer'], # , 'Washing Machine' --> consumes energy constantly; , 'Dishwasher' --> noise at 3am
    6: ['Washing Machine', 'Dishwasher'],
    7: ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
    8: ['Washing Machine'], # 'Dryer' --> consumes constantly
    9: ['Washer Dryer', 'Washing Machine', 'Dishwasher'], 
    10: ['Washing Machine'] #'Dishwasher'
}

<br>

**Active Appliances**

In [ ]:
# manual input, determined using information provided in the readme
# validated in the next step using the validate thresholds functionality
active_appliances = {
    1: deepcopy(shiftable_devices[1]) + ['Television Site', 'Computer Site'],
    2: deepcopy(shiftable_devices[2]) + ['Television', 'Microwave', 'Toaster', 'Hi-Fi', 'Kettle'],
    3: deepcopy(shiftable_devices[3]) + ['Toaster', 'Television', 'Microwave', 'Kettle'],
    4: deepcopy(shiftable_devices[4]) + ['Television Site', 'Kettle'], #'Microwave', 'Computer Site' --> consume energy constantly 
    5: deepcopy(shiftable_devices[5]) + ['Television Site', 'Combination Microwave', 'Kettle', 'Toaster'], # 'Computer Site', --> consumes energy constantly
    6: deepcopy(shiftable_devices[6]) + ['MJY Computer', 'Kettle', 'Toaster'], #, 'PGM Computer', 'Television Site' 'Microwave' --> consume energy constantly 
    7: deepcopy(shiftable_devices[7]) + ['Television Site', 'Toaster', 'Kettle'],
    8: deepcopy(shiftable_devices[8]) + ['Toaster', 'Kettle'], # 'Television Site', 'Computer' --> consume energy constantly
    9: deepcopy(shiftable_devices[9]) + ['Microwave', 'Kettle'], #'Television Site', 'Hi-Fi' --> consume energy constantly
    10: deepcopy(shiftable_devices[10]) + ['Magimix (Blender)', 'Microwave'] # 'Television Site' --> consume energy constantly
}

<br>

**Energy Consumption Threshold**

Our Preparation Agent will require the energy consumption threshold, which will determine if a device was used in a given period. This threshold will allow to reduce the impact of noise in the data. We will determine the optimal thresholds for the households using the Preparation Agent’s validate thresholds method. To demonstrate how noise in the consumption data occurs, we call the validate thresholds method for household 1. The consumption data regarding the Television Site in household 1 seems to contain daily noise around 3 am. We will choose the optimal threshold, such that the noise is removed from the data.

Furthermore, we will create our initial evaluation configuration file for each household which will contain the specified user input.

In [ ]:
# validating the thresholds for household 1 to show noise in the data
household_id = 1

In [ ]:
# creating the config including the user input
config =  {'data': {'household': deepcopy(household_id)}}
config['user_input'] = {
    'shiftable_devices': deepcopy(shiftable_devices[config['data']['household']]),
    'active_appliances': deepcopy(active_appliances[config['data']['household']])
}

# initializing the evaluation agent
model_type = "logit"

In [ ]:
evaluation = Performance_Evaluation_Agent(DATA_PATH, model_type, config, load_data=True)

In [ ]:
preparation = evaluation.preparation

# Data-Preparation
df_th = preparation.truncate(preparation.input)
df_th = preparation.scale(df_th)
df_th = helper.aggregate(df_th, '60T')

# Graphical analysis of candidate thresholds
#thresholds = [0] + list(np.geomspace(.01, .4, 5))
#preparation.validate_thresholds(df_th, thresholds, config['user_input']['active_appliances'])

In [ ]:
thresholds = {
    1: 0.15,
    2: 0.01,
    3: 0.01, 
    4: 0.01, 
    5: 0.025,
    6: 0.065, 
    7: 0.01, 
    8: 0.01, # washing machine over night
    9: 0.01, 
    10: 0.01
}

<br>

### **2.2 Running our Pipeline**

Before we are able to analyze the performance of our recommender system, we need to calculate all outputs of all our agents and all recommendations possible based on the available data. To conveniently compute these outputs and recommendations for the households, we added a pipeline method to the Evaluation Agent which allows to run every agent of our recommender system for every available date iteratively. We will demonstrate its functionality by creating the recommendations for household 3. 

Additionally, we will use a further method of the Evaluation Agent to receive the default configuration for evaluating our recommender system.

In [ ]:
household_id = 1

In [ ]:
# creating the config including the user input
config =  {'data': {'household': deepcopy(household_id)}}
config['user_input'] = {
    'shiftable_devices': deepcopy(shiftable_devices[config['data']['household']]),
    'active_appliances': deepcopy(active_appliances[config['data']['household']]),
    'threshold': deepcopy(thresholds[config['data']['household']])
}

<br>

**Preparing the data**

In [ ]:
# calling the evaluation agent
evaluation = Performance_Evaluation_Agent(DATA_PATH, model_type, config)
evaluation.get_default_config('preparation')
evaluation.config

In [ ]:
evaluation.pipeline('preparation')

In [ ]:
evaluation.df['activity'][100:105]

In [ ]:
evaluation.df['usage'][100:105]

In [ ]:
evaluation.df['load'][335:340]

<br>

**Creating all recommendations**


In [ ]:
evaluation.get_default_config(['activity', 'usage', 'load'])
evaluation.config

In [ ]:
activity_predictions = evaluation._pipeline_activity_usage_load("activity")

In [ ]:
#this takes the prediction out of nesting and puts it into own row
activity_predictions_test = activity_predictions.explode('Prediction')
activity_predictions_test

In [ ]:
# just to create time (other way?)
row = 1
hour = 0
hours = []
while row <=len(activity_predictions_test):
    hours.append(hour)
    hour += 1
    row += 1
    if hour == 24:
        hour = 0

print(hours)

In [21]:
activity_predictions_test['time'] = hours
activity_predictions_test

,Agent,Date,Time,Prediction,time
10,activity,2013-10-23,<module 'time' (built-in)>,0.000011,0
10,activity,2013-10-23,<module 'time' (built-in)>,0.000016,1
10,activity,2013-10-23,<module 'time' (built-in)>,0.000024,2
10,activity,2013-10-23,<module 'time' (built-in)>,0.000035,3
10,activity,2013-10-23,<module 'time' (built-in)>,0.000051,4
...,...,...,...,...,...
634,activity,2015-07-09,<module 'time' (built-in)>,0.731011,19
634,activity,2015-07-09,<module 'time' (built-in)>,0.743255,20
634,activity,2015-07-09,<module 'time' (built-in)>,0.755129,21
634,activity,2015-07-09,<module 'time' (built-in)>,0.766626,22
